In [1]:
import pyspark.pandas as ps
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pandas as pd

/home/himedia/anaconda3/envs/spark/lib/python3.11/site-packages/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/08 15:17:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.csv("./data/Taitanic_train.csv", header=True)

In [4]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [5]:
data.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
data = data.withColumn('family', data['Sibsp'] + data['Parch'])

# 필요없는 컬럼 drop

In [7]:
data = data.drop('PassengerId', 'Name', 'Sibsp', 'parch', 'Ticket')

In [8]:
data.show()

+--------+------+------+----+-------+-----+--------+------+
|Survived|Pclass|   Sex| Age|   Fare|Cabin|Embarked|family|
+--------+------+------+----+-------+-----+--------+------+
|       0|     3|  male|  22|   7.25| null|       S|   1.0|
|       1|     1|female|  38|71.2833|  C85|       C|   1.0|
|       1|     3|female|  26|  7.925| null|       S|   0.0|
|       1|     1|female|  35|   53.1| C123|       S|   1.0|
|       0|     3|  male|  35|   8.05| null|       S|   0.0|
|       0|     3|  male|null| 8.4583| null|       Q|   0.0|
|       0|     1|  male|  54|51.8625|  E46|       S|   0.0|
|       0|     3|  male|   2| 21.075| null|       S|   4.0|
|       1|     3|female|  27|11.1333| null|       S|   2.0|
|       1|     2|female|  14|30.0708| null|       C|   1.0|
|       1|     3|female|   4|   16.7|   G6|       S|   2.0|
|       1|     1|female|  58|  26.55| C103|       S|   0.0|
|       0|     3|  male|  20|   8.05| null|       S|   0.0|
|       0|     3|  male|  39| 31.275| nu

In [9]:
data.columns

['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Cabin', 'Embarked', 'family']

# 결측값 찾기

In [10]:
for c in data.columns:
    print(c)

Survived
Pclass
Sex
Age
Fare
Cabin
Embarked
family


In [11]:
data.select(f.col("Age").isNull().cast("int")).show()

+--------------------------+
|CAST((Age IS NULL) AS INT)|
+--------------------------+
|                         0|
|                         0|
|                         0|
|                         0|
|                         0|
|                         1|
|                         0|
|                         0|
|                         0|
|                         0|
|                         0|
|                         0|
|                         0|
|                         0|
|                         0|
|                         0|
|                         0|
|                         1|
|                         0|
|                         1|
+--------------------------+
only showing top 20 rows



In [12]:
data.select(*(f.sum(f.col(c).isNull().cast("int")).alias(c) for c in data.columns)).show()

+--------+------+---+---+----+-----+--------+------+
|Survived|Pclass|Sex|Age|Fare|Cabin|Embarked|family|
+--------+------+---+---+----+-----+--------+------+
|       0|     0|  0|177|   0|  687|       2|     0|
+--------+------+---+---+----+-----+--------+------+



# 결측값 처리

In [13]:
data.na.drop().toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  183 non-null    object 
 1   Pclass    183 non-null    object 
 2   Sex       183 non-null    object 
 3   Age       183 non-null    object 
 4   Fare      183 non-null    object 
 5   Cabin     183 non-null    object 
 6   Embarked  183 non-null    object 
 7   family    183 non-null    float64
dtypes: float64(1), object(7)
memory usage: 11.6+ KB


In [14]:
data.na.drop(subset='Age').toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  714 non-null    object 
 1   Pclass    714 non-null    object 
 2   Sex       714 non-null    object 
 3   Age       714 non-null    object 
 4   Fare      714 non-null    object 
 5   Cabin     185 non-null    object 
 6   Embarked  712 non-null    object 
 7   family    714 non-null    float64
dtypes: float64(1), object(7)
memory usage: 44.8+ KB


In [15]:
data.na.drop(subset=['Age','Embarked']).toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    object 
 1   Pclass    712 non-null    object 
 2   Sex       712 non-null    object 
 3   Age       712 non-null    object 
 4   Fare      712 non-null    object 
 5   Cabin     183 non-null    object 
 6   Embarked  712 non-null    object 
 7   family    712 non-null    float64
dtypes: float64(1), object(7)
memory usage: 44.6+ KB


In [16]:
data.drop('Cabin').na.drop().toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    object 
 1   Pclass    712 non-null    object 
 2   Sex       712 non-null    object 
 3   Age       712 non-null    object 
 4   Fare      712 non-null    object 
 5   Embarked  712 non-null    object 
 6   family    712 non-null    float64
dtypes: float64(1), object(6)
memory usage: 39.1+ KB


# 결측값 대치

In [17]:
# f.mean 평균
data.select(f.mean('Age')).show()

+-----------------+
|         avg(Age)|
+-----------------+
|29.69911764705882|
+-----------------+



In [18]:
data.na.fill('29', 'Age')

DataFrame[Survived: string, Pclass: string, Sex: string, Age: string, Fare: string, Cabin: string, Embarked: string, family: double]

In [19]:
data.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    object 
 1   Pclass    891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    object 
 4   Fare      891 non-null    object 
 5   Cabin     204 non-null    object 
 6   Embarked  889 non-null    object 
 7   family    891 non-null    float64
dtypes: float64(1), object(7)
memory usage: 55.8+ KB


# imputer를 사용한 결측값 처리

In [20]:
data.printSchema()

root
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- family: double (nullable = true)



In [21]:
data.select(f.sum(data['Age'].isNull().cast('int'))).show()

+-------------------------------+
|sum(CAST((Age IS NULL) AS INT))|
+-------------------------------+
|                            177|
+-------------------------------+



In [22]:
data = data.withColumn('Age', data['Age'].cast('Integer'))
data = data.withColumn('Survived', data['Survived'].cast('Integer'))
data = data.withColumn('Pclass', data['Pclass'].cast('Integer'))
data = data.withColumn('family', data['family'].cast('int'))

In [23]:
data.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- family: integer (nullable = true)



In [24]:
from pyspark.ml.feature import Imputer

In [25]:
# setStrategy('mean', 'median', 'mode')
imputer = Imputer(inputCols=['Age'], outputCols=['Age']).setStrategy('median')
imputer.fit(data).transform(data).show()

+--------+------+------+---+-------+-----+--------+------+
|Survived|Pclass|   Sex|Age|   Fare|Cabin|Embarked|family|
+--------+------+------+---+-------+-----+--------+------+
|       0|     3|  male| 22|   7.25| null|       S|     1|
|       1|     1|female| 38|71.2833|  C85|       C|     1|
|       1|     3|female| 26|  7.925| null|       S|     0|
|       1|     1|female| 35|   53.1| C123|       S|     1|
|       0|     3|  male| 35|   8.05| null|       S|     0|
|       0|     3|  male| 28| 8.4583| null|       Q|     0|
|       0|     1|  male| 54|51.8625|  E46|       S|     0|
|       0|     3|  male|  2| 21.075| null|       S|     4|
|       1|     3|female| 27|11.1333| null|       S|     2|
|       1|     2|female| 14|30.0708| null|       C|     1|
|       1|     3|female|  4|   16.7|   G6|       S|     2|
|       1|     1|female| 58|  26.55| C103|       S|     0|
|       0|     3|  male| 20|   8.05| null|       S|     0|
|       0|     3|  male| 39| 31.275| null|       S|     

In [26]:
# 성별별, Pclass별 생존자 수 구하기
data.groupby('Sex', 'Pclass').agg({'Survived' : 'sum'}).show()

+------+------+-------------+
|   Sex|Pclass|sum(Survived)|
+------+------+-------------+
|  male|     3|           47|
|female|     3|           72|
|female|     1|           91|
|female|     2|           70|
|  male|     2|           17|
|  male|     1|           45|
+------+------+-------------+



In [27]:
data.show()

+--------+------+------+----+-------+-----+--------+------+
|Survived|Pclass|   Sex| Age|   Fare|Cabin|Embarked|family|
+--------+------+------+----+-------+-----+--------+------+
|       0|     3|  male|  22|   7.25| null|       S|     1|
|       1|     1|female|  38|71.2833|  C85|       C|     1|
|       1|     3|female|  26|  7.925| null|       S|     0|
|       1|     1|female|  35|   53.1| C123|       S|     1|
|       0|     3|  male|  35|   8.05| null|       S|     0|
|       0|     3|  male|null| 8.4583| null|       Q|     0|
|       0|     1|  male|  54|51.8625|  E46|       S|     0|
|       0|     3|  male|   2| 21.075| null|       S|     4|
|       1|     3|female|  27|11.1333| null|       S|     2|
|       1|     2|female|  14|30.0708| null|       C|     1|
|       1|     3|female|   4|   16.7|   G6|       S|     2|
|       1|     1|female|  58|  26.55| C103|       S|     0|
|       0|     3|  male|  20|   8.05| null|       S|     0|
|       0|     3|  male|  39| 31.275| nu

In [28]:
data = data.drop('Fare', 'Cabin')
data.show()

+--------+------+------+----+--------+------+
|Survived|Pclass|   Sex| Age|Embarked|family|
+--------+------+------+----+--------+------+
|       0|     3|  male|  22|       S|     1|
|       1|     1|female|  38|       C|     1|
|       1|     3|female|  26|       S|     0|
|       1|     1|female|  35|       S|     1|
|       0|     3|  male|  35|       S|     0|
|       0|     3|  male|null|       Q|     0|
|       0|     1|  male|  54|       S|     0|
|       0|     3|  male|   2|       S|     4|
|       1|     3|female|  27|       S|     2|
|       1|     2|female|  14|       C|     1|
|       1|     3|female|   4|       S|     2|
|       1|     1|female|  58|       S|     0|
|       0|     3|  male|  20|       S|     0|
|       0|     3|  male|  39|       S|     6|
|       0|     3|female|  14|       S|     0|
|       1|     2|female|  55|       S|     0|
|       0|     3|  male|   2|       Q|     5|
|       1|     2|  male|null|       S|     0|
|       0|     3|female|  31|     

In [29]:
data = data.na.fill(29, 'Age')

In [30]:
data.toPandas()['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [31]:
data = data.na.fill('S', 'Embarked')

In [32]:
data.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- family: integer (nullable = true)



# 레이블 인코딩

In [33]:
# 레이블 인코딩 하기 scikit-learn의 label encoding과 같은 역할
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCols=['Sex', 'Embarked'], outputCols=["SexIndex", "EmbarkedIndex"])
data = indexer.fit(data).transform(data)

In [34]:
data.show()

+--------+------+------+---+--------+------+--------+-------------+
|Survived|Pclass|   Sex|Age|Embarked|family|SexIndex|EmbarkedIndex|
+--------+------+------+---+--------+------+--------+-------------+
|       0|     3|  male| 22|       S|     1|     0.0|          0.0|
|       1|     1|female| 38|       C|     1|     1.0|          1.0|
|       1|     3|female| 26|       S|     0|     1.0|          0.0|
|       1|     1|female| 35|       S|     1|     1.0|          0.0|
|       0|     3|  male| 35|       S|     0|     0.0|          0.0|
|       0|     3|  male| 29|       Q|     0|     0.0|          2.0|
|       0|     1|  male| 54|       S|     0|     0.0|          0.0|
|       0|     3|  male|  2|       S|     4|     0.0|          0.0|
|       1|     3|female| 27|       S|     2|     1.0|          0.0|
|       1|     2|female| 14|       C|     1|     1.0|          1.0|
|       1|     3|female|  4|       S|     2|     1.0|          0.0|
|       1|     1|female| 58|       S|     0|    

# One-hot 인코딩

In [35]:
# 원핫인코딩 하기
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols=["SexIndex", "EmbarkedIndex"], outputCols=['Sex_vec', 'Embarked_vec'])
data = encoder.fit(data).transform(data)

In [36]:
data.show()

+--------+------+------+---+--------+------+--------+-------------+-------------+-------------+
|Survived|Pclass|   Sex|Age|Embarked|family|SexIndex|EmbarkedIndex|      Sex_vec| Embarked_vec|
+--------+------+------+---+--------+------+--------+-------------+-------------+-------------+
|       0|     3|  male| 22|       S|     1|     0.0|          0.0|(1,[0],[1.0])|(2,[0],[1.0])|
|       1|     1|female| 38|       C|     1|     1.0|          1.0|    (1,[],[])|(2,[1],[1.0])|
|       1|     3|female| 26|       S|     0|     1.0|          0.0|    (1,[],[])|(2,[0],[1.0])|
|       1|     1|female| 35|       S|     1|     1.0|          0.0|    (1,[],[])|(2,[0],[1.0])|
|       0|     3|  male| 35|       S|     0|     0.0|          0.0|(1,[0],[1.0])|(2,[0],[1.0])|
|       0|     3|  male| 29|       Q|     0|     0.0|          2.0|(1,[0],[1.0])|    (2,[],[])|
|       0|     1|  male| 54|       S|     0|     0.0|          0.0|(1,[0],[1.0])|(2,[0],[1.0])|
|       0|     3|  male|  2|       S|   

In [37]:
data = data.drop('Sex', 'Embarked', 'SexIndex', 'EmbarkedIndex')

In [38]:
data.show()

+--------+------+---+------+-------------+-------------+
|Survived|Pclass|Age|family|      Sex_vec| Embarked_vec|
+--------+------+---+------+-------------+-------------+
|       0|     3| 22|     1|(1,[0],[1.0])|(2,[0],[1.0])|
|       1|     1| 38|     1|    (1,[],[])|(2,[1],[1.0])|
|       1|     3| 26|     0|    (1,[],[])|(2,[0],[1.0])|
|       1|     1| 35|     1|    (1,[],[])|(2,[0],[1.0])|
|       0|     3| 35|     0|(1,[0],[1.0])|(2,[0],[1.0])|
|       0|     3| 29|     0|(1,[0],[1.0])|    (2,[],[])|
|       0|     1| 54|     0|(1,[0],[1.0])|(2,[0],[1.0])|
|       0|     3|  2|     4|(1,[0],[1.0])|(2,[0],[1.0])|
|       1|     3| 27|     2|    (1,[],[])|(2,[0],[1.0])|
|       1|     2| 14|     1|    (1,[],[])|(2,[1],[1.0])|
|       1|     3|  4|     2|    (1,[],[])|(2,[0],[1.0])|
|       1|     1| 58|     0|    (1,[],[])|(2,[0],[1.0])|
|       0|     3| 20|     0|(1,[0],[1.0])|(2,[0],[1.0])|
|       0|     3| 39|     6|(1,[0],[1.0])|(2,[0],[1.0])|
|       0|     3| 14|     0|   

# 독립변수를 하나의 벡터로 만들기 Features

In [39]:
data.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- family: integer (nullable = true)
 |-- Sex_vec: vector (nullable = true)
 |-- Embarked_vec: vector (nullable = true)



In [40]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols = ['Pclass', 'Age', 'family', 'Sex_vec', 'Embarked_vec'], outputCol="Features")
data = feature_assembler.transform(data)
data.show()

+--------+------+---+------+-------------+-------------+--------------------+
|Survived|Pclass|Age|family|      Sex_vec| Embarked_vec|            Features|
+--------+------+---+------+-------------+-------------+--------------------+
|       0|     3| 22|     1|(1,[0],[1.0])|(2,[0],[1.0])|[3.0,22.0,1.0,1.0...|
|       1|     1| 38|     1|    (1,[],[])|(2,[1],[1.0])|[1.0,38.0,1.0,0.0...|
|       1|     3| 26|     0|    (1,[],[])|(2,[0],[1.0])|[3.0,26.0,0.0,0.0...|
|       1|     1| 35|     1|    (1,[],[])|(2,[0],[1.0])|[1.0,35.0,1.0,0.0...|
|       0|     3| 35|     0|(1,[0],[1.0])|(2,[0],[1.0])|[3.0,35.0,0.0,1.0...|
|       0|     3| 29|     0|(1,[0],[1.0])|    (2,[],[])|[3.0,29.0,0.0,1.0...|
|       0|     1| 54|     0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,54.0,0.0,1.0...|
|       0|     3|  2|     4|(1,[0],[1.0])|(2,[0],[1.0])|[3.0,2.0,4.0,1.0,...|
|       1|     3| 27|     2|    (1,[],[])|(2,[0],[1.0])|[3.0,27.0,2.0,0.0...|
|       1|     2| 14|     1|    (1,[],[])|(2,[1],[1.0])|[2.0,14.

In [41]:
data = data.drop('Pclass', 'Age', 'family', 'Sex_vec', 'Embarked_vec')
data.show()

+--------+--------------------+
|Survived|            Features|
+--------+--------------------+
|       0|[3.0,22.0,1.0,1.0...|
|       1|[1.0,38.0,1.0,0.0...|
|       1|[3.0,26.0,0.0,0.0...|
|       1|[1.0,35.0,1.0,0.0...|
|       0|[3.0,35.0,0.0,1.0...|
|       0|[3.0,29.0,0.0,1.0...|
|       0|[1.0,54.0,0.0,1.0...|
|       0|[3.0,2.0,4.0,1.0,...|
|       1|[3.0,27.0,2.0,0.0...|
|       1|[2.0,14.0,1.0,0.0...|
|       1|[3.0,4.0,2.0,0.0,...|
|       1|[1.0,58.0,0.0,0.0...|
|       0|[3.0,20.0,0.0,1.0...|
|       0|[3.0,39.0,6.0,1.0...|
|       0|[3.0,14.0,0.0,0.0...|
|       1|[2.0,55.0,0.0,0.0...|
|       0|[3.0,2.0,5.0,1.0,...|
|       1|[2.0,29.0,0.0,1.0...|
|       0|[3.0,31.0,1.0,0.0...|
|       1|[3.0,29.0,0.0,0.0...|
+--------+--------------------+
only showing top 20 rows



In [43]:
data.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Features: vector (nullable = true)



# 트레인과 테스트로 홀드아웃

In [44]:
(train_data, test_data) = data.randomSplit([0.8, 0.2], seed = 10)

In [45]:
test_data.show()

+--------+--------------------+
|Survived|            Features|
+--------+--------------------+
|       0|(6,[0,1],[3.0,21.0])|
|       0|(6,[0,1],[3.0,29.0])|
|       0|[1.0,2.0,3.0,0.0,...|
|       0|[1.0,21.0,1.0,1.0...|
|       0|[1.0,24.0,1.0,1.0...|
|       0|[1.0,28.0,0.0,1.0...|
|       0|[1.0,29.0,0.0,1.0...|
|       0|[1.0,29.0,1.0,1.0...|
|       0|[1.0,31.0,1.0,1.0...|
|       0|[1.0,33.0,0.0,1.0...|
|       0|[1.0,45.0,0.0,1.0...|
|       0|[1.0,45.0,0.0,1.0...|
|       0|[1.0,46.0,0.0,1.0...|
|       0|[1.0,46.0,1.0,1.0...|
|       0|[1.0,50.0,1.0,1.0...|
|       0|[1.0,52.0,2.0,1.0...|
|       0|[1.0,54.0,1.0,1.0...|
|       0|[1.0,55.0,0.0,1.0...|
|       0|[1.0,56.0,0.0,1.0...|
|       0|[1.0,62.0,0.0,1.0...|
+--------+--------------------+
only showing top 20 rows



# 의사 결정 나무로 분석하기

In [46]:
from pyspark.ml.classification import DecisionTreeClassifier

In [47]:
dtc = DecisionTreeClassifier(featuresCol= 'Features', labelCol='Survived', maxDepth=3)
dtc = dtc.fit(train_data)
pred = dtc.transform(test_data)
pred.show()

+--------+--------------------+-------------+--------------------+----------+
|Survived|            Features|rawPrediction|         probability|prediction|
+--------+--------------------+-------------+--------------------+----------+
|       0|(6,[0,1],[3.0,21.0])|  [38.0,53.0]|[0.41758241758241...|       1.0|
|       0|(6,[0,1],[3.0,29.0])|  [38.0,53.0]|[0.41758241758241...|       1.0|
|       0|[1.0,2.0,3.0,0.0,...|  [7.0,128.0]|[0.05185185185185...|       1.0|
|       0|[1.0,21.0,1.0,1.0...|  [58.0,34.0]|[0.63043478260869...|       0.0|
|       0|[1.0,24.0,1.0,1.0...|  [58.0,34.0]|[0.63043478260869...|       0.0|
|       0|[1.0,28.0,0.0,1.0...|  [58.0,34.0]|[0.63043478260869...|       0.0|
|       0|[1.0,29.0,0.0,1.0...|  [58.0,34.0]|[0.63043478260869...|       0.0|
|       0|[1.0,29.0,1.0,1.0...|  [58.0,34.0]|[0.63043478260869...|       0.0|
|       0|[1.0,31.0,1.0,1.0...|  [58.0,34.0]|[0.63043478260869...|       0.0|
|       0|[1.0,33.0,0.0,1.0...|  [58.0,34.0]|[0.63043478260869..

In [48]:
from sklearn.metrics import accuracy_score, classification_report

In [49]:
pred = pred.toPandas()

In [51]:
pred

,Survived,Features,rawPrediction,probability,prediction
0,0,"(3.0, 21.0, 0.0, 0.0, 0.0, 0.0)","[38.0, 53.0]","[0.4175824175824176, 0.5824175824175825]",1.0
1,0,"(3.0, 29.0, 0.0, 0.0, 0.0, 0.0)","[38.0, 53.0]","[0.4175824175824176, 0.5824175824175825]",1.0
2,0,"[1.0, 2.0, 3.0, 0.0, 1.0, 0.0]","[7.0, 128.0]","[0.05185185185185185, 0.9481481481481482]",1.0
3,0,"[1.0, 21.0, 1.0, 1.0, 1.0, 0.0]","[58.0, 34.0]","[0.6304347826086957, 0.3695652173913043]",0.0
4,0,"[1.0, 24.0, 1.0, 1.0, 0.0, 1.0]","[58.0, 34.0]","[0.6304347826086957, 0.3695652173913043]",0.0
...,...,...,...,...,...
182,1,"[3.0, 29.0, 2.0, 1.0, 0.0, 1.0]","[305.0, 41.0]","[0.8815028901734104, 0.11849710982658959]",0.0
183,1,"[3.0, 30.0, 0.0, 1.0, 1.0, 0.0]","[305.0, 41.0]","[0.8815028901734104, 0.11849710982658959]",0.0
184,1,"[3.0, 31.0, 0.0, 1.0, 1.0, 0.0]","[305.0, 41.0]","[0.8815028901734104, 0.11849710982658959]",0.0
185,1,"[3.0, 32.0, 0.0, 1.0, 1.0, 0.0]","[305.0, 41.0]","[0.8815028901734104, 0.11849710982658959]",0.0


In [52]:
print(accuracy_score(pred['prediction'], pred['Survived']))

0.8342245989304813


/home/himedia/anaconda3/envs/spark/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/himedia/anaconda3/envs/spark/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/himedia/anaconda3/envs/spark/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/himedia/anaconda3/envs/spark/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(d

In [53]:
print(classification_report(pred['prediction'], pred['Survived']))

              precision    recall  f1-score   support

         0.0       0.90      0.84      0.87       122
         1.0       0.74      0.82      0.77        65

    accuracy                           0.83       187
   macro avg       0.82      0.83      0.82       187
weighted avg       0.84      0.83      0.84       187



/home/himedia/anaconda3/envs/spark/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/himedia/anaconda3/envs/spark/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/himedia/anaconda3/envs/spark/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/himedia/anaconda3/envs/spark/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(d